In [1]:
from QHyper.solvers.quantum_annealing.advantage import Advantage

import numpy as np
import networkx as nx
from collections import defaultdict

from QHyper.converter import Converter
from QHyper.constraint import Polynomial

from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
from dimod.sampleset import SampleSet

Some util function from QHyper

In [2]:
def convert_qubo_keys(qubo: Polynomial) -> tuple[dict[tuple, float], float]:
    new_qubo = defaultdict(float)
    offset = 0.0

    qubo, offset = qubo.separate_const()
    for k, v in qubo.terms.items():
        if len(k) == 1:
            new_key = (k[0], k[0])
        elif len(k) > 2:
            raise ValueError("Only supports quadratic model")
        else:
            new_key = k

        new_qubo[new_key] += v

    return (new_qubo, offset)

Load our biggest graph

In [3]:
Graphs = np.load("networks/powerlaw_m=1_p=0.2/graphs.npy", allow_pickle=True)
G = Graphs[9]

Simple binary clustering of this graph

In [4]:
from QHyper.problems.community_detection import CommunityDetectionProblem, Network


network = Network(graph=G)
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

Get the standard Advantage sampler and embedding composite

In [5]:
sampler = DWaveSampler(solver="Advantage_system5.4", region="eu-central-1")
embedding_composite = EmbeddingComposite(sampler)

In [6]:
qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)

In [7]:
%time sampleset = EmbeddingComposite(sampler).sample(bqm, num_reads=100)

CPU times: total: 1min 9s
Wall time: 1min 38s


In [8]:
res = sampleset.first.sample

c0 = sorted([int(k[1:]) for k, v in res.items() if v == 0])
c1 = sorted([int(k[1:]) for k, v in res.items() if v == 1])

# Some simple intuition on the results
print("c0 len:", len(c0))
print("c1 len:", len(c1))
print("Q: ", nx.community.modularity(G, [c0, c1]))

c0 len: 47
c1 len: 53
Q:  0.45913682277318646


Each time the .sample method is called on the sampler, a new embedding is calculated.\
Previously calculated embeddings can be extracted from the sampleset info.

In [12]:
sampleset = embedding_composite.sample(
    bqm, num_reads=100, return_embedding=True
)

In [13]:
embedding_from_sampleset = sampleset.info["embedding_context"]["embedding"]
embedding_from_sampleset

{'x1': (4430,
  4429,
  1821,
  4428,
  4427,
  4431,
  4432,
  4433,
  4434,
  1823,
  1822,
  4089,
  4359),
 'x0': (1328, 1329, 1327, 1330, 1326, 4942, 4943, 1331, 1325, 1324),
 'x2': (4024, 4023, 775, 771, 772, 4027, 4025, 773, 4026, 4028, 774),
 'x3': (1478,
  3635,
  1477,
  1476,
  1475,
  1474,
  3698,
  1479,
  1480,
  1481,
  1482,
  3637,
  3636,
  1023,
  1024),
 'x4': (1507,
  1506,
  1505,
  1508,
  1509,
  1510,
  1511,
  1504,
  3952,
  3953,
  1503,
  1512),
 'x5': (1164,
  1165,
  4971,
  1256,
  5001,
  5002,
  1163,
  1162,
  1161,
  5003,
  1160,
  4895),
 'x6': (4325,
  4321,
  4326,
  4327,
  4328,
  4329,
  4324,
  4323,
  4322,
  517,
  516,
  518,
  519),
 'x7': (4235, 4236, 4237, 4238, 4234, 4233, 577, 4232, 4239, 576, 578),
 'x8': (4443,
  848,
  4534,
  4535,
  4536,
  4537,
  4538,
  4442,
  4441,
  4440,
  4444,
  398,
  397,
  396),
 'x9': (1418, 1417, 1419, 1420, 1416, 1415, 4582, 4583, 4581, 4580, 4579),
 'x10': (4085, 4086, 4087, 4088, 4084, 4083, 846

Let's find embedding the same way it is done automatically in EmbeddingComposite class \
(code extracted from the EmbeddingComposite class)

In [14]:
import minorminer
import dimod


# Extracted from EmbeddingComposite class sourcecode
find_embedding=minorminer.find_embedding
child_structure_search = dimod.child_structure_dfs

In [15]:
# Extracted from EmbeddincComposite class sourcecode

# apply the embedding to the given problem to map it to the child sampler
target_structure = child_structure_search(sampler)
__, target_edgelist, target_adjacency = target_structure

# add self-loops to edgelist to handle singleton variables
source_edgelist = list(bqm.quadratic) + [(v, v) for v in bqm.linear]

In [16]:
%time embedding_calculated = find_embedding(source_edgelist, target_edgelist)

CPU times: total: 1min 25s
Wall time: 1min 47s


In [18]:
embedding_calculated["x3"]

[4188,
 4463,
 4189,
 1748,
 1749,
 1750,
 1751,
 1752,
 1753,
 4462,
 1493,
 1492,
 4192,
 4191,
 4190,
 1418,
 1754]

In [19]:
embedding_from_sampleset["x3"]

(1478,
 3635,
 1477,
 1476,
 1475,
 1474,
 3698,
 1479,
 1480,
 1481,
 1482,
 3637,
 3636,
 1023,
 1024)

The minorminer.find_embedding default method is heuristic, \
I'll set a seed to see if the embedding we calculate aside with find_embedding is same as the embedding calculated automatically with EmbeddingComposite

In [20]:
sampleset_with_random_seed = embedding_composite.sample(
    bqm, num_reads=100, return_embedding=True, embedding_parameters={"random_seed": 10}
)

In [21]:
embedding_calculated_rs = find_embedding(source_edgelist, target_edgelist, random_seed=10)

In [22]:
embedding_from_sampleset_rs = sampleset_with_random_seed.info["embedding_context"]["embedding"]

No surprise, they're the same:

In [23]:
sorted(embedding_calculated_rs) == sorted(embedding_from_sampleset_rs)

True

Let's put the calculated embedding to FixedEmbeddingComposite

In [24]:
from dwave.system.composites import FixedEmbeddingComposite


fixed_embedding_composite = FixedEmbeddingComposite(sampler, embedding=embedding_calculated_rs)

Let's sample from it

In [25]:
%time sampleset_fe = fixed_embedding_composite.sample(bqm, num_reads=100, return_embedding=True)

CPU times: total: 46.9 ms
Wall time: 121 ms


In [26]:
res_fe = sampleset_fe.first.sample

# Again, some simple results intuition
c0_fe = sorted([int(k[1:]) for k, v in res_fe.items() if v == 0])
c1_fe = sorted([int(k[1:]) for k, v in res_fe.items() if v == 1])

print("c0_fe len:", len(c0_fe))
print("c1_fe len:", len(c1_fe))
print("Sample modularity from EmbeddingComposite:", nx.community.modularity(G, [c0, c1]))
print("Sample modularity from FixedEmbeddingComposite:", nx.community.modularity(G, [c0_fe, c1_fe]))

c0_fe len: 48
c1_fe len: 52
Sample modularity from EmbeddingComposite: 0.45913682277318646
Sample modularity from FixedEmbeddingComposite: 0.4393429241914091


Let's see how much time can be saved when we do minorminer.find_embedding parallelly

In [28]:
from joblib import Parallel, delayed


N_RUNS = 8
n_jobs = 8

%time embeddings = Parallel(n_jobs=n_jobs)(delayed(find_embedding)(source_edgelist, target_edgelist, random_seed=10) for _ in range(N_RUNS))

CPU times: total: 46.9 ms
Wall time: 4min 55s


In [29]:
def run_standard_loop():
    embeddings_2 = []

    for _ in range(N_RUNS):
        emb = find_embedding(source_edgelist, target_edgelist, random_seed=10)
        embeddings_2.append(emb)

%time run_standard_loop()

CPU times: total: 8min 54s
Wall time: 20min 43s


Ineed some time can be saved by paralellizing embedding calculations.

Let's now imagine we go down in hierarchy (hierarchical search) \
and see how time needed to find embedding scales with subgraphs of smaller sizes.\
Let's divide c0:

In [30]:
problem_c0 = CommunityDetectionProblem(network_data=Network(graph=G, community=c0), communities=2, one_hot_encoding=False)
qubo_c0 = Converter.create_qubo(problem_c0, [])
qubo_terms_c0, offset_c0 = convert_qubo_keys(qubo_c0)
bqm_c0 = BinaryQuadraticModel.from_qubo(qubo_terms_c0, offset=offset_c0)

In [31]:
source_edgelist_c0 = list(bqm_c0.quadratic) + [(v, v) for v in bqm_c0.linear]
%time embeddings_c0 = Parallel(n_jobs=n_jobs)(delayed(find_embedding)(source_edgelist_c0, target_edgelist, random_seed=10) for _ in range(N_RUNS))

CPU times: total: 62.5 ms
Wall time: 16.8 s


In [32]:
def run_standard_loop():
    embeddings_2_c0 = []
    for _ in range(N_RUNS):
        emb = find_embedding(source_edgelist_c0, target_edgelist, random_seed=10)
        embeddings_2_c0.append(emb)

%time run_standard_loop()

CPU times: total: 35 s
Wall time: 1min 21s


_____________

___________

Last example to show sampling with a fixed embedding is quick (bqm for the full graph of 100 nodes):

In [33]:
def sample_fe():
    sample = FixedEmbeddingComposite(sampler, embedding=embedding_calculated_rs).sample(bqm, num_reads=100)
    return sample.first.sample

def sample_fe_loop():
    results = []
    for _ in range(N_RUNS):
        results.append(sample_fe())
    # return results

%time sample_fe_loop()

CPU times: total: 1.17 s
Wall time: 29.3 s


_____________

_____________

Comparison with dwave.embedding.pegasus find_clique_embedding

In [8]:
from dwave.embedding.pegasus import find_clique_embedding
import dwave_networkx as dnx

In [9]:
bqm_graph = bqm.to_networkx_graph()

C:\Users\basia\AppData\Local\Temp\ipykernel_22728\3242365722.py:1: DeprecationWarning: BinaryQuadraticModel.to_networkx_graph() is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Use dimod.to_networkx_graph() instead.
  bqm_graph = bqm.to_networkx_graph()


In [10]:
from minorminer.busclique import busgraph_cache

busgraph_cache.clear_all_caches()

In [11]:
%time emb = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 1min 41s
Wall time: 2min 24s


Embeddings already calculated are cached with busgraph_cache:

In [12]:
%time emb = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 31.2 ms
Wall time: 85.5 ms


In [17]:
from dwave.system.composites import FixedEmbeddingComposite


sample_cl = FixedEmbeddingComposite(sampler, embedding=emb).sample(bqm, num_reads=100)

res_cl = sample_cl.first.sample

c0_cl = sorted([int(k[1:]) for k, v in res_cl.items() if v == 0])
c1_cl = sorted([int(k[1:]) for k, v in res_cl.items() if v == 1])

print("c0_cl len:", len(c0_cl))
print("c1_cl len:", len(c1_cl))
# print("Sample modularity from EmbeddingComposite:", nx.community.modularity(G, [c0, c1]))
print("Sample modularity from FixedEmbeddingComposite:", nx.community.modularity(G, [c0_cl, c1_cl]))

c0_cl len: 49
c1_cl len: 51
Sample modularity from FixedEmbeddingComposite: 0.4696969696969697
